In [7]:
# -------------------------------
# Importuri
# -------------------------------
import re
import string
import numpy as np
import pandas as pd
import joblib

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [6]:
# -------------------------------
# Setup NLTK
# -------------------------------
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:

# -------------------------------
# Preconfigurare: stop words & stemmer
# -------------------------------
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# -------------------------------
# Funcție de preprocesare
# -------------------------------
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    lower = text.lower()
    nopunct = [c for c in lower if c not in string.punctuation] #1.
    nopunct = ''.join(nopunct)
    # Alternativ, poți folosi regex dacă vrei să păstrezi $ și !:
    # nopunct = re.sub(r"[^a-z\s$!]", "", lower)
    tokens = word_tokenize(nopunct)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return " ".join(tokens)

# -------------------------------
# Încărcare și preprocesare dataset
# -------------------------------
df = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=["label", "message"])
df["message"] = df["message"].apply(preprocess_text)
y = df["label"].apply(lambda x: 1 if x == "spam" else 0)

In [12]:
# -------------------------------
# Split: train/test
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(df["message"], y, test_size=0.2, random_state=42)

In [13]:
y_train

1978    1
3989    0
3935    0
4078    0
4086    1
       ..
3772    0
5191    0
5226    0
5390    0
860     0
Name: label, Length: 4457, dtype: int64

In [14]:
# Exemplu de text (mesaje) cu CountVectorizer si TFIDFTRANSFORMER
df = pd.DataFrame({
    'message': [
        "Win money now",
        "Win a free prize now"
    ]
})

# Crează CountVectorizer cu bigrame
# un min-df mai mic adauga cuvinte care apar foarte rar
# un max_df mai mare de 90% va permite cuvintelor care sunt mai frecvente în setul de date să rămână în vocabular.
# ngram_range=(1, 2) face unigram si bigram. Cu (1,3) face si trigram
vectorizer = CountVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))

# Obține matricea de frecvențe
X_counts = vectorizer.fit_transform(df["message"])
# Afișează matricea de frecvențe completă
X_counts_array = X_counts.toarray()

# Obține termenii (unigramele și bigramele)
df_terms = vectorizer.get_feature_names_out()

# Creează un DataFrame pentru a vizualiza frecvențele pentru primele 5 mesaje
df_subset = pd.DataFrame(X_counts_array, columns=df_terms)

# Afișează DataFrame-ul cu frecvențele
print(df_subset)

# Aplică TfidfTransformer pentru a transforma matricea de frecvențe într-o matrice TF-IDF
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=df_terms)

# Afișează DataFrame-ul cu scorurile TF-IDF
print(df_tfidf)

   free  free prize  money  money now  prize  prize now  win free  win money
0     0           0      1          1      0          0         0          1
1     1           1      0          0      1          1         1          0
       free  free prize    money  money now     prize  prize now  win free  \
0  0.000000    0.000000  0.57735    0.57735  0.000000   0.000000  0.000000   
1  0.447214    0.447214  0.00000    0.00000  0.447214   0.447214  0.447214   

   win money  
0    0.57735  
1    0.00000  


In [15]:
# -------------------------------
# Pipeline și GridSearch
# -------------------------------
# -------------------------------
# Pipeline cu [TfidfVectorizer] ORI cu [CountVectorizer] ORI cu [CountVectorizer+ TFID]
# -------------------------------
#CountVectorizer: transformă textul în frecvențe brute (cât de des apare un cuvânt/bigram/etc.).
#TfidfTransformer: transformă acele frecvențe în scoruri TF-IDF, care penalizează cuvintele comune și dau mai multă greutate celor rare și informative.

#vectorizer = TfidfVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))
vectorizer = CountVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))

pipeline = Pipeline([ #Ordinea este importantă: vectorizer ➜ tfidf ➜ classifier.
    ("vectorizer", vectorizer),
    ("tfidf", TfidfTransformer()),
    #("classifier", MultinomialNB()),
    #("classifier", LogisticRegression(solver="saga", max_iter=5000))  # liblinear e bun pentru small/medium datasets
    #("classifier", LinearSVC())
    ("classifier", RandomForestClassifier(random_state=42))
])
# ======================================================================================
# param grid pt MultinomialNB
#param_grid = {"classifier__alpha": [0.01, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 1.0]}
# ======================================================================================
# Param grid pt Logistic Regression
#param_grid = {
#    "classifier__C": [0.01, 0.1, 1, 10],  # C = inverse regularization strength (cu cât e mai mic, cu atât regularizează mai puternic)
#    "classifier__penalty": ["l1", "l2"]
#}
# =======================================================================================
# Parametri pentru SVM
#param_grid = {
#    "classifier__C": [0.1, 1, 10],
#    "vectorizer__min_df": [1],
#    "vectorizer__max_df": [0.9],
#}
# ========================================================================================
# Parametri pt RandomForest (poți ajusta mai mulți dacă vrei)
param_grid = {
    "classifier__n_estimators": [50, 100, 150],
    "classifier__max_depth": [None, 10, 20],
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,#cross-validation Modelul este antrenat pe 4 din cele 5 părți și testat pe cea de-a 5-a parte (foldul de testare).
    scoring="f1"
)
### fara test_train split aplic training direct pe coloana mesage
#grid_search.fit(df["message"], y)

### cu train test split
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print("Best model parameters:", grid_search.best_params_)

# ERROR on LOGISTIC REGRESSION
#!!!!! ConvergenceWarning: Liblinear failed to converge, increase the number of iterations. warnings.warn(!!!! (1000 este by default)
# Metoda 1 -> creste nr de iteratii: LogisticRegression(solver="liblinear", max_iter=5000)
# Metoda 2 -> incearca alt solver: LogisticRegression(solver="saga", max_iter=1000) ... saga e optimizat pentru dataset-uri mari și pentru L1/L2 penalty.
# Metoda 3 -> Ajustează C (regularizarea): "classifier__C": [0.1, 1, 10, 50] 
# Metoda 4 -> De obicei, Logistic Regression funcționează mai bine dacă faci scalare. Adaugă un StandardScaler înainte de classifier:
# pipeline = Pipeline([
#    ("vectorizer", CountVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))),
#    ("scaler", StandardScaler(with_mean=False)),  # Normalizează datele
#    ("classifier", LogisticRegression(solver="liblinear", max_iter=1000))
#])

Best model parameters: {'classifier__max_depth': None, 'classifier__n_estimators': 150}


# Intelegere Grid-Search
pipeline = Pipeline([

    ("vvvv", CountVectorizer()),
    ("cccc", MultinomialNB())
])
# MultinomialNB 
– parametri principali care pot fi ajustati de Grid Search: MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

Detalii despre fiecare parametru:

1️⃣ - alpha (default = 1.0)

    Este smoothing-ul Laplace (sau Lidstone).
    Evită împărțirea la 0 când un cuvânt nu apare în training.
    Valori mai mici = mai puțin smoothing (ex: alpha=0.1).
    Valori mai mari = model mai conservator.

✅ Este cel mai important de reglat

2️⃣ - fit_prior (default = True)
    
    Decide dacă să estimeze automat distribuția prior a claselor (spam vs. ham).
    Dacă False, presupune că toate clasele au probabilitate egală, indiferent de câte exemple sunt.

3️⃣ - class_prior (default = None)
    
    Dacă fit_prior=False, poți seta manual tu distribuția claselor.
    Ex: [0.7, 0.3] dacă știi că ai 70% ham și 30% spam.
    Nu e folosit des, dar poate fi util în contexte specifice.

Dacă vrei să ajustezi parametrii din MultinomialNB, scrii: 

param_grid = {

    "cccc__alpha": [0.01, 0.1, 0.5, 1.0],
    "cccc__fit_prior": [True, False]
}

Dacă vrei să reglezi CountVectorizer, scrii:

param_grid = {

    "vvvv__ngram_range": [(1, 1), (1, 2)],
    "vvvv__max_df": [0.75, 0.9, 1.0]
}

In [16]:
print(grid_search.best_params_)
print(grid_search.best_score_) # best F1 score
# Scoruri pentru toate combinațiile:
cv_results = pd.DataFrame(grid_search.cv_results_)
print(cv_results[["params", "mean_test_score", "rank_test_score"]])
# Dacă vrei să vezi toți parametrii care pot fi tunati într-un Pipeline
pipeline.get_params().keys()

{'classifier__max_depth': None, 'classifier__n_estimators': 150}
0.8249466328308589
                                              params  mean_test_score  \
0  {'classifier__max_depth': None, 'classifier__n...         0.809376   
1  {'classifier__max_depth': None, 'classifier__n...         0.824786   
2  {'classifier__max_depth': None, 'classifier__n...         0.824947   
3  {'classifier__max_depth': 10, 'classifier__n_e...         0.148095   
4  {'classifier__max_depth': 10, 'classifier__n_e...         0.165022   
5  {'classifier__max_depth': 10, 'classifier__n_e...         0.151088   
6  {'classifier__max_depth': 20, 'classifier__n_e...         0.528262   
7  {'classifier__max_depth': 20, 'classifier__n_e...         0.515433   
8  {'classifier__max_depth': 20, 'classifier__n_e...         0.508137   

   rank_test_score  
0                3  
1                2  
2                1  
3                9  
4                7  
5                8  
6                4  
7                

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'vectorizer', 'tfidf', 'classifier', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__monotonic_cst', 'classifier__n_

In [17]:
# -------------------------------
# Evaluare pe setul de test daca folosesc train_test
# -------------------------------
y_pred = best_model.predict(X_test)
print("\n--- Evaluation on Test Set ---")
print(classification_report(y_test, y_pred, target_names=["Not-Spam", "Spam"]))


--- Evaluation on Test Set ---
              precision    recall  f1-score   support

    Not-Spam       0.97      1.00      0.98       966
        Spam       1.00      0.79      0.88       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115



# Intelegere metrics
# Când îți pasă mai mult să detectezi toate Spamurile (adică vrei recall bun)
Ai 100 de mesaje, din care: 20 sunt Spam, 80 sunt Not-Spam! Modelul tău clasifică: 18 ca fiind Spam si 82 ca Not-Spam.

Dintre cele 18: 15 sunt corect clasificate ca Spam (True Positives ✅), 3 sunt greșit clasificate ca Spam, dar sunt de fapt Not-Spam (False Positives ❌). Și a ratat 5 mesaje Spam, spunând că sunt Not-Spam (False Negatives ❌)

Termen          |      Ce e                      | Valoare

True Positives  | Spam detectat corect           | 15

False Positives | Not-Spam clasificat ca Spam    | 3

False Negatives | Spam ratat (zice că nu e spam) | 5

Calcule:

    Precision = Ce procent din mesajele prezise ca Spam erau chiar Spam?

#Precision = TP / (TP+FP) = 15 / (15+3) = 15 / 18 = 0.83

    Recall = Ce procent din mesajele reale Spam au fost detectate?

#Recall = TP / TP+FN = 15 / 15+5 = 15 / 20 = 0.75

Precision	--> „Din ce am zis că e spam, câte erau cu adevărat?”	Când vrei puține alarme false (ex. emailuri bune marcate ca spam)

Recall	    --> „Din toate spamurile reale, câte am prins?”	Când vrei să ratezi cât mai puține spamuri (ex. filtru agresiv)

# F1-score (media armonică dintre precision și recall) 

Adică:
F1 = 2 ⋅ (precision⋅recall) / (precision+recall

F1 = 2⋅(0.83+0.75) / (0.83⋅0.75) ≈ 0.79

Intuiție:

    Dacă precision este mare, dar recall e mic → Modelul e „prea timid”. Nu face multe greșeli, dar ratează spamuri.

    Dacă recall este mare, dar precision e mic → Modelul e „prea agresiv”. Prinde aproape toate spamurile, dar alertează greșit și emailuri bune.

    F1-score echilibrează ambele. E util mai ales când ai date dezechilibrate (ex: mult mai multe Not-Spam decât Spam).

# Accuracy (Procentul de predicții corecte din total)
Accuracy = (TP+TN) / (TP+TN+FP+FN)



In [18]:
# -------------------------------
# Mesaje de test și predicții
# -------------------------------
new_messages = [
    "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/1234 to claim now.",
    "Hey, are we still meeting up for lunch today?",
    "Urgent! Your account has been compromised. Verify your details here: www.fakebank.com/verify",
    "Reminder: Your appointment is scheduled for tomorrow at 10am.",
    "FREE entry in a weekly competition to win an iPad. Just text WIN to 80085 now!",
]

processed_messages = [preprocess_text(msg) for msg in new_messages]
X_new = best_model.named_steps["vectorizer"].transform(processed_messages)

predictions = best_model.named_steps["classifier"].predict(X_new)
prediction_probabilities = best_model.named_steps["classifier"].predict_proba(X_new)

for i, msg in enumerate(new_messages):
    prediction = "Spam" if predictions[i] == 1 else "Not-Spam"
    spam_probability = prediction_probabilities[i][1]
    ham_probability = prediction_probabilities[i][0]

    print(f"Message: {msg}")
    print(f"Prediction: {prediction}")
    print(f"Spam Probability: {spam_probability:.2f}")
    print(f"Not-Spam Probability: {ham_probability:.2f}")
    print("-" * 50)

Message: Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/1234 to claim now.
Prediction: Not-Spam
Spam Probability: 0.48
Not-Spam Probability: 0.52
--------------------------------------------------
Message: Hey, are we still meeting up for lunch today?
Prediction: Not-Spam
Spam Probability: 0.00
Not-Spam Probability: 1.00
--------------------------------------------------
Message: Urgent! Your account has been compromised. Verify your details here: www.fakebank.com/verify
Prediction: Not-Spam
Spam Probability: 0.16
Not-Spam Probability: 0.84
--------------------------------------------------
Message: Reminder: Your appointment is scheduled for tomorrow at 10am.
Prediction: Not-Spam
Spam Probability: 0.02
Not-Spam Probability: 0.98
--------------------------------------------------
Message: FREE entry in a weekly competition to win an iPad. Just text WIN to 80085 now!
Prediction: Spam
Spam Probability: 0.52
Not-Spam Probability: 0.48
--------------------------

In [19]:
########## SPECIAL PENTRU SVM #################
new_messages = [
    "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/1234 to claim now.",
    "Hey, are we still meeting up for lunch today?",
    "Urgent! Your account has been compromised. Verify your details here: www.fakebank.com/verify",
    "Reminder: Your appointment is scheduled for tomorrow at 10am.",
    "FREE entry in a weekly competition to win an iPad. Just text WIN to 80085 now!",
]

processed_messages = [preprocess_text(msg) for msg in new_messages]
X_new = best_model.named_steps["vectorizer"].transform(processed_messages)

predictions = best_model.named_steps["classifier"].predict(X_new)
scores = best_model.named_steps["classifier"].decision_function(X_new)

for i, msg in enumerate(new_messages):
    prediction = "Spam" if predictions[i] == 1 else "Not-Spam"
    confidence = scores[i]

    print(f"Message: {msg}")
    print(f"Prediction: {prediction}")
    print(f"Confidence Score: {confidence:.2f}")  # mai mare => mai sigur
    print("-" * 50)


AttributeError: 'RandomForestClassifier' object has no attribute 'decision_function'

In [7]:
# -------------------------------
# Salvare model
# -------------------------------
model_filename = 'spam_detection_model.joblib'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

Model saved to spam_detection_model.joblib


In [38]:
# loaded_model = joblib.load(model_filename)
# predictions = loaded_model.predict(new_messages)